## Here I translate GCN in tensorflow to GCN in tensorflow.keras


In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras import layers

In [2]:
# tf and tf keras versions
print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


In [8]:
# load the input data and test set

adj = np.load('/Users/b_eebs/tf-keras/ZINC/adj/0.npy').astype('float32')
features = np.load('/Users/b_eebs/tf-keras/ZINC/features/0.npy').astype('float32')
test = (np.load('/Users/b_eebs/tf-keras/ZINC/logP.npy')[0:10000]).astype('float32') #takes the first 10,000 molecules


In [12]:
features_ = np.array_split(features,100)
test_ = np.array_split(test,100)
adj_ = np.array_split(adj,100)
print(features_[0].shape,adj_[0].shape, test_[0].shape)

(100, 50, 58) (100, 50, 50) (100,)


In [13]:
# here I define the GCN layers

import conv_models
import tensorflow.keras.backend as K

class convLayers(layers.Layer):
    models = ['gcn', 'gcn_a', 'gcn_g', 'gcn_ag']
    

    def __init__(self,adjacency,features,output_dim, 
                 weight_shape,bias_shape, model, **kwargs):
        self.output_dim = output_dim 
        self.weight_shape = weight_shape
        self.bias_shape = bias_shape
        self.model = model
        self.features = K.variable(features, 
                                       dtype = tf.float32)
        self.adjacency = K.variable(adjacency, 
                                        dtype = tf.float32)
        super(convLayers, self).__init__(**kwargs)
    
    # here are the weight and bias for the layer
    # still have to add attention weights
    # initializers may be incorrect
    def build(self,input_shape):
        self.kernel = self.add_weight(name='kernel',
                                  shape=self.weight_shape,
                                  initializer='glorot_uniform',
                                  trainable=True,dtype=tf.float32)
        
        self.bias = self.add_weight(name='bias',
                                shape=self.bias_shape,
                                initializer='glorot_uniform',
                                trainable=True,dtype=tf.float32)
        super(convLayers, self).build(input_shape)
    
    # here is where I will implement all models, which are built
    # in conv_models.py. 
    def call(self, inputs):
        if self.model == convLayers.models[0]:
            print('go to gcn function and calc. mult.')
            _features = conv_models.GCN(self.features,self.adjacency,
                                        self.kernel,self.bias)
            print(_features)
        elif self.model == convLayers.models[1]:
            print('go to gcn+att function and calc. mult.')
        elif self.model == convLayers.models[2]:
            print('go to gcn+gate function and calc. mult.')
        elif self.model == convLayers.models[3]:
            print('go to aug. gcn function and calc. mult.')
        else:
            raise ValueError('not a valid model')
        return _features

    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape) 

In [17]:
conv_output = 32
weight_shape = [features.shape[2],conv_output]
K.set_learning_phase(1)
# define the model, composed of one layer defined in convLayers class
model = tf.keras.Sequential([convLayers(adj[0:100],features[0:100],
                            conv_output,weight_shape,[conv_output],'gcn'), 
                            layers.Activation('relu')],name = 'input layer')
# model compiler that optimizes and monters the mse and mae per batch
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='mse',
              metrics=['mae'])


# fit
# ((10000, 50, 58) , (10000,))
model.fit(features[0:100], test[0:100])





go to gcn function and calc. mult.
Tensor("MatMul_4:0", shape=(100, 50, 32), dtype=float32)


InvalidArgumentError: Incompatible shapes: [100,50,32] vs. [32,1]
	 [[{{node training_6/Adam/gradients/loss_4/output_1_loss/MeanSquaredError/sub_grad/BroadcastGradientArgs}}]]